In [3]:
import pandas as pd

accidents = pd.read_csv("data/raw/accidents_2019_2023.csv")

/var/folders/05/_vxzr9653lg058wd1tp1rdyr0000gn/T/ipykernel_78881/1844118324.py:3: DtypeWarning: Columns (42,45,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents = pd.read_csv("data/raw/accidents_2019_2023.csv")


In [6]:
accidents['lat'] = accidents['lat'].str.replace(',', '.')
accidents['long'] = accidents['long'].str.replace(',', '.')

accidents['lat'] = accidents['lat'].astype(float)
accidents['long'] = accidents['long'].astype(float)

In [7]:
accidents['grav'] = accidents['grav'].replace({2: 42})
accidents['grav'] = accidents['grav'].replace({4: 2})
accidents['grav'] = accidents['grav'].replace({42: 4})

In [8]:
accidents_copy = accidents.drop(['occutc','secu2','secu3','locp','actp','etatp','v1','v2','vosp','lartpc','larrout'],axis=1)

In [16]:
accidents_copy['age'] = accidents_copy['an']-accidents_copy['an_nais']
accidents_copy.drop(['an_nais'],axis=1,inplace = True)

In [14]:
accidents_copy = accidents_copy[accidents_copy['grav'] != -1]

In [18]:
for col in accidents_copy.columns:
    print(f"\n{col} :")
    print(accidents_copy[col].unique(),accidents_copy[col].nunique())


Num_Acc :
[201900000001 201900000002 201900000003 ... 202300054820 202300054821
 202300054822] 273226

id_vehicule :
['138\xa0306\xa0524' '138\xa0306\xa0525' '138\xa0306\xa0523' ...
 '155\xa0583\xa0347' '155\xa0583\xa0344' '155\xa0583\xa0345'] 462904

num_veh :
['B01' 'A01' 'C01' 'D01' 'E01' 'Z01' 'Y01' 'I01' 'T01' 'O01' 'G01' 'F01'
 'M01' 'LB01' 'H01' 'J01' 'K01' 'L01' 'N01' 'W01' 'X01' 'U01' 'V01' 'Q01'
 'MA01' 'CB01' '\\01' 'VF01' 'P01' 'LA01' 'AA01' 'NA01' 'SA01' 'CA01'
 'BA01' 'AB01' 'PA01' 'EB01' 'WB01' 'RA01' 'MV01' 'EA01' 'TB01' 'AY01'
 'SB01' 'HB01' 'FA01' 'BB01' 'VB01' 'GA01' 'MB01' 'PB01' '[01' 'IB01'
 'DA01' 'DB01' 'HA01' 'XA01' 'TA01' 'ZZ01' 'GB01' 'IA01' 'FB01' 'YA01'
 'KA01' 'KD01' 'R01' 'S01' ']01' 'WX01' 'XB01' 'JA01' 'RB01' 'VA01' 'UB01'
 'OB01' 'ZA01' 'AC01' 'TC01' 'OA01' '_01'] 81

senc :
[ 2  1  3  0 -1] 5

catv :
[ 7 17 33 42 30 37 32 50 38 10  1 40 15 14 99  2 80 34 60 31  0 21  3 13
 20 43 36 39 16 35 41 -1] 32

obs :
[ 0  1  4 14  9  6 15 13  8  2 16 12  3  7 